Working with Neural Network Models

© Hans Nieminen, Satakunta University of Applied Sciences

# Exercise 9.1

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from itertools import product

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## Data

In [ ]:
# Load the Wine Quality dataset
data = pd.read_csv("https://raw.githubusercontent.com/haniemi/deeplearning/main/data/winequality_red.csv", delimiter=';')

In [ ]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [ ]:
data["quality"].value_counts()

,count
quality,
5,681
6,638
7,199
4,53
8,18
3,10


In [ ]:
# Separate features and target
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [ ]:
X[:5]

array([[7.400e+00, 7.000e-01, 0.000e+00, 1.900e+00, 7.600e-02, 1.100e+01,
        3.400e+01, 9.978e-01, 3.510e+00, 5.600e-01, 9.400e+00],
       [7.800e+00, 8.800e-01, 0.000e+00, 2.600e+00, 9.800e-02, 2.500e+01,
        6.700e+01, 9.968e-01, 3.200e+00, 6.800e-01, 9.800e+00],
       [7.800e+00, 7.600e-01, 4.000e-02, 2.300e+00, 9.200e-02, 1.500e+01,
        5.400e+01, 9.970e-01, 3.260e+00, 6.500e-01, 9.800e+00],
       [1.120e+01, 2.800e-01, 5.600e-01, 1.900e+00, 7.500e-02, 1.700e+01,
        6.000e+01, 9.980e-01, 3.160e+00, 5.800e-01, 9.800e+00],
       [7.400e+00, 7.000e-01, 0.000e+00, 1.900e+00, 7.600e-02, 1.100e+01,
        3.400e+01, 9.978e-01, 3.510e+00, 5.600e-01, 9.400e+00]])

In [ ]:
# Standardize the X dataset
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
y[:5]

array([5, 5, 5, 6, 5])

In [ ]:
# Encode labels to integers starting from 0
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [ ]:
y[:5]

array([2, 2, 2, 3, 2])

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=8)

In [ ]:
# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, X, y, device='cpu'):
      self.X = X.to(device)
      self.y = y.to(device)

    def __len__(self):
      return len(self.y)

    def __getitem__(self, idx):
      return self.X[idx], self.y[idx]

In [ ]:
# Create a custom dataset
train_dataset = CustomDataset(X_train, y_train, device)

In [ ]:
# Create a dataloader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)

## Neural network

In [ ]:
# Define the neural network
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(11,64)
        nn.init.kaiming_normal_(self.layer1.weight,
                                nonlinearity='relu')
        nn.init.zeros_(self.layer1.bias)
        self.layer1_act = nn.ReLU()
        self.layer2 = nn.Linear(64, 32)
        nn.init.kaiming_normal_(self.layer2.weight,
                                nonlinearity='relu')
        nn.init.zeros_(self.layer2.bias)
        self.layer2_act = nn.ReLU()
        self.layer3 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.layer1_act(self.layer1(x))
        x = self.layer2_act(self.layer2(x))
        x = self.layer3(x)
        return x

In [ ]:
# Create the model
torch.manual_seed(99)
model = NeuralNetwork().to(device)

In [ ]:
# Train the model
num_epochs = 30

In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Training

In [ ]:
# Training loop
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, targets in train_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

Epoch 1/30, Loss: 1.9965
Epoch 2/30, Loss: 1.2876
Epoch 3/30, Loss: 1.1005
Epoch 4/30, Loss: 1.0360
Epoch 5/30, Loss: 1.0001
Epoch 6/30, Loss: 0.9753
Epoch 7/30, Loss: 0.9560
Epoch 8/30, Loss: 0.9402
Epoch 9/30, Loss: 0.9266
Epoch 10/30, Loss: 0.9146
Epoch 11/30, Loss: 0.9033
Epoch 12/30, Loss: 0.8929
Epoch 13/30, Loss: 0.8830
Epoch 14/30, Loss: 0.8738
Epoch 15/30, Loss: 0.8650
Epoch 16/30, Loss: 0.8562
Epoch 17/30, Loss: 0.8478
Epoch 18/30, Loss: 0.8393
Epoch 19/30, Loss: 0.8319
Epoch 20/30, Loss: 0.8244
Epoch 21/30, Loss: 0.8171
Epoch 22/30, Loss: 0.8103
Epoch 23/30, Loss: 0.8033
Epoch 24/30, Loss: 0.7966
Epoch 25/30, Loss: 0.7903
Epoch 26/30, Loss: 0.7839
Epoch 27/30, Loss: 0.7780
Epoch 28/30, Loss: 0.7718
Epoch 29/30, Loss: 0.7658
Epoch 30/30, Loss: 0.7604


### Evaluating

In [ ]:
# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    outputs = model(X_test.to(device))
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test.to(device)).sum().item() / y_test.size(0)
    print(f'Test Accuracy: {accuracy:.4f}')

Test Accuracy: 0.6031


### Hyperparameter tuning

In [ ]:
# Hyperparameter tuning
learning_rates = [0.0005, 0.001, 0.01, 0.1]
betas = [(0.9, 0.999), (0.95, 0.999), (0.99, 0.999), (0.9, 0.990), (0.9, 0.985), (0.9, 0.98)]

In [ ]:
best_accuracy = 0
best_params = None

In [ ]:
for lr, beta in product(learning_rates, betas):
    torch.manual_seed(99)
    model = NeuralNetwork().to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, betas=beta)

    # Training loop
    model.train()
    for epoch in range(num_epochs):
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

    # Evaluate the model
    model.eval()
    with torch.no_grad():
        outputs = model(X_test.to(device))
        _, predicted = torch.max(outputs, 1)
        accuracy = (predicted == y_test.to(device)).sum().item() / y_test.size(0)
        print(f'Learning Rate: {lr}, Betas: {beta}, Test Accuracy: {accuracy:.4f}')

        # Check if we have a new best accuracy
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = (lr, beta)

print(f'Best Hyperparameters - Learning Rate: {best_params[0]}, Betas: {best_params[1]}, Accuracy: {best_accuracy:.4f}')

Learning Rate: 0.0005, Betas: (0.9, 0.999), Test Accuracy: 0.5875
Learning Rate: 0.0005, Betas: (0.95, 0.999), Test Accuracy: 0.5906
Learning Rate: 0.0005, Betas: (0.99, 0.999), Test Accuracy: 0.5906
Learning Rate: 0.0005, Betas: (0.9, 0.99), Test Accuracy: 0.5906
Learning Rate: 0.0005, Betas: (0.9, 0.985), Test Accuracy: 0.5844
Learning Rate: 0.0005, Betas: (0.9, 0.98), Test Accuracy: 0.5844
Learning Rate: 0.001, Betas: (0.9, 0.999), Test Accuracy: 0.6031
Learning Rate: 0.001, Betas: (0.95, 0.999), Test Accuracy: 0.6125
Learning Rate: 0.001, Betas: (0.99, 0.999), Test Accuracy: 0.6125
Learning Rate: 0.001, Betas: (0.9, 0.99), Test Accuracy: 0.6000
Learning Rate: 0.001, Betas: (0.9, 0.985), Test Accuracy: 0.5969
Learning Rate: 0.001, Betas: (0.9, 0.98), Test Accuracy: 0.6000
Learning Rate: 0.01, Betas: (0.9, 0.999), Test Accuracy: 0.5531
Learning Rate: 0.01, Betas: (0.95, 0.999), Test Accuracy: 0.6094
Learning Rate: 0.01, Betas: (0.99, 0.999), Test Accuracy: 0.6188
Learning Rate: 0.01,